<a href="https://colab.research.google.com/github/michaelarjelm/evaluacionTresIA/blob/main/Evaluacion_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Estamos implementando un avanzado sistema de Smart City para transformar Ancud en una ciudad inteligente. Ancud, una pintoresca ciudad ubicada en el norte de la Isla de Chiloé en Chile, es conocida por su rica historia, hermosos paisajes y vibrante cultura local. Este sistema integrará tecnologías de última generación para optimizar y mejorar diversos aspectos de la vida urbana, incluyendo la gestión del tráfico, la administración del suministro de agua, y la infraestructura pública. Utilizando técnicas de inteligencia artificial, control difuso, y teoría de juegos de Nash, nuestro objetivo es crear una ciudad más eficiente, segura y sostenible, mejorando así la calidad de vida de sus habitantes y posicionando a Ancud como un modelo de innovación y modernidad en la región.






# Importar bibliotecas necesarias



In [34]:
!pip install scikit-fuzzy # Instala la biblioteca scikit-fuzzy, que proporciona herramientas para trabajar con lógica difusa.
!pip install nashpy # Instala la biblioteca nashpy, que permite trabajar con teoría de juegos y encontrar equilibrios de Nash.
!pip install networkx # Instala la biblioteca networkx, que proporciona herramientas para la creación, manipulación y estudio de la estructura, dinámica y funciones de redes complejas.
!pip install sympy # Instala la biblioteca sympy, que es una biblioteca para cálculos simbólicos en Python.
import ipywidgets as widgets # Importa la biblioteca ipywidgets para crear widgets interactivos en Jupyter notebooks.
from IPython.display import display # Importa la función display de IPython.display para mostrar widgets en la salida del notebook.
import numpy as np # Importa la biblioteca numpy para operaciones numéricas avanzadas.
import skfuzzy as fuzz # Importa la biblioteca scikit-fuzzy para trabajar con lógica difusa.
from skfuzzy import control as ctrl # Importa el módulo control de scikit-fuzzy para crear sistemas de control difusos.
from datetime import datetime # Importa la clase datetime de la biblioteca datetime para manejar y manipular fechas y horas.
import networkx as nx # Importa la biblioteca networkx para crear y manejar grafos y redes.
from sympy import symbols, And, Implies, Not, satisfiable # Importa funciones y símbolos de la biblioteca sympy para cálculos simbólicos.
from skfuzzy import control as ctrl # Importa el módulo control de scikit-fuzzy para crear sistemas de control difusos.
from datetime import datetime # Importa la clase datetime de la biblioteca datetime para manejar y manipular fechas y horas.
import networkx as nx  # Importa la biblioteca networkx para crear y manejar grafos y redes.
import nashpy as nash  # Importa la biblioteca nashpy para trabajar con teoría de juegos y encontrar equilibrios de Nash.


# Creación Widgets para generar casos.

Este código crea una interfaz interactiva en un notebook de Jupyter utilizando widgets para simular y gestionar un sistema de control de tráfico y un sistema de gestión de agua. Los usuarios pueden ajustar diferentes parámetros como la hora del día, el nivel de tráfico y la condición climática para ver cómo afectan la intensidad de las luces. También pueden ajustar el estado del suministro de agua, las válvulas, las bombas y los sensores para ver cómo estos cambios afectan el sistema de agua. Además, hay un selector para el estado del semáforo.

Casos de uso.

* Hora: 18:00 (hora punta)
* Tráfico: 80%
* Clima: Desfavorable
* Madrugada con tráfico bajo y clima favorable:

---------------------------------------------------------------------

* Hora: 3:00
* Tráfico: 10%
* Clima: Favorable
* Estado crítico del suministro de agua:

----------------------------------------------------------------------

* Estado del suministro: sin agua
* Válvulas: cerradas
* Bombas: no funcionando
* Sensores: no funcionando
* Condiciones normales durante el día:

----------------------------------------------------------------------

* Hora: 12:00
* Tráfico: 50%
* Clima: Normal
* Estado del suministro: normal
* Válvulas: abiertas
* Bombas: funcionando
* Sensores: funcionando



In [35]:
# Parámetros configurables para el sistema de control de tráfico
# Crea un slider para seleccionar la hora del día.
hora_slider = widgets.IntSlider(value=12, min=0, max=23, step=1, description='Hora:')

# Crea un slider para seleccionar el nivel de tráfico en porcentaje.
trafico_slider = widgets.IntSlider(value=50, min=0, max=100, step=10, description='Tráfico (%):')

# Crea un desplegable para seleccionar la condición climática.
clima_dropdown = widgets.Dropdown(
    options=['Favorable', 'Normal', 'Desfavorable'],
    value='Normal',
    description='Clima:',
)

# Muestra los widgets de hora, tráfico y clima en la salida del notebook.
display(hora_slider, trafico_slider, clima_dropdown)

# Parámetros configurables para el sistema de gestión de agua
# Crea un desplegable para seleccionar el estado del suministro de agua.
estado_suministro_selector = widgets.Dropdown(
    options=['normal', 'baja presión', 'sin agua'],
    value='normal',
    description='Estado suministro:'
)

# Crea un diccionario de widgets desplegables para seleccionar el estado de las válvulas.
estado_valvulas_selectors = {
    'valvula1': widgets.Dropdown(options=['abierta', 'cerrada'], value='abierta', description='Válvula 1:'),
    'valvula2': widgets.Dropdown(options=['abierta', 'cerrada'], value='abierta', description='Válvula 2:')
}

# Crea un diccionario de widgets desplegables para seleccionar el estado de las bombas.
estado_bombas_selectors = {
    'bomba1': widgets.Dropdown(options=['funcionando', 'no funcionando'], value='funcionando', description='Bomba 1:'),
    'bomba2': widgets.Dropdown(options=['funcionando', 'no funcionando'], value='funcionando', description='Bomba 2:')
}

# Crea un diccionario de widgets desplegables para seleccionar el estado de los sensores.
estado_sensores_selectors = {
    'sensor1': widgets.Dropdown(options=['funcionando', 'no funcionando'], value='funcionando', description='Sensor 1:'),
    'sensor2': widgets.Dropdown(options=['funcionando', 'no funcionando'], value='funcionando', description='Sensor 2:')
}

# Muestra el widget del estado del suministro de agua en la salida del notebook.
display(estado_suministro_selector)

# Muestra los widgets del estado de las válvulas en la salida del notebook.
for w in estado_valvulas_selectors.values():
    display(w)

# Muestra los widgets del estado de las bombas en la salida del notebook.
for w in estado_bombas_selectors.values():
    display(w)

# Muestra los widgets del estado de los sensores en la salida del notebook.
for w in estado_sensores_selectors.values():
    display(w)

# Crea un desplegable para seleccionar el estado del semáforo.
semaforo_selector = widgets.Dropdown(
    options=['verde', 'amarillo', 'rojo'],
    value='verde',
    description='Estado semáforo:'
)

# Muestra el widget del estado del semáforo en la salida del notebook.
display(semaforo_selector)


IntSlider(value=12, description='Hora:', max=23)

IntSlider(value=50, description='Tráfico (%):', step=10)

Dropdown(description='Clima:', index=1, options=('Favorable', 'Normal', 'Desfavorable'), value='Normal')

Dropdown(description='Estado suministro:', options=('normal', 'baja presión', 'sin agua'), value='normal')

Dropdown(description='Válvula 1:', options=('abierta', 'cerrada'), value='abierta')

...

Dropdown(description='Bomba 1:', options=('funcionando', 'no funcionando'), value='funcionando')

Dropdown(description='Bomba 2:', options=('funcionando', 'no funcionando'), value='funcionando')

Dropdown(description='Sensor 1:', options=('funcionando', 'no funcionando'), value='funcionando')

Dropdown(description='Sensor 2:', options=('funcionando', 'no funcionando'), value='funcionando')

Dropdown(description='Estado semáforo:', options=('verde', 'amarillo', 'rojo'), value='verde')

# Algoritmo Genético

Este código implementa un algoritmo genético para optimizar la distribución de autobuses en varias zonas. Define diferentes zonas y asigna costos y eficiencias a cada una. El objetivo es encontrar la mejor distribución de autobuses que maximice la eficiencia y minimice los costos.

El proceso comienza creando una población inicial de soluciones posibles. Luego, el algoritmo evalúa qué tan buena es cada solución (su "fitness") y selecciona las mejores para la siguiente generación. Las mejores soluciones se cruzan y mutan para crear nuevas soluciones. Este ciclo de evaluación, selección, cruce y mutación se repite durante varias generaciones para mejorar las soluciones gradualmente.

In [36]:

# Definición de las zonas a considerar en el problema.
zonas = ["Centro", "Villa Chacao", "Pudeto", "Fátima", "Quetalmahue", "Pumillahue", "Lechagua", "Calen", "Pugueñún", "Caulín"]

# Costos asociados a cada zona, almacenados en un array numpy.
costos = np.array([10, 8, 9, 7, 6, 5, 8, 7, 9, 6])

# Eficiencia asociada a cada zona, almacenada en un array numpy.
eficiencia = np.array([8, 9, 7, 8, 6, 9, 7, 5, 8, 7])

# Función para calcular el "fitness" (aptitud) de una distribución dada.
def calcular_fitness(distribucion):
    # Calcula el costo total multiplicando la distribución por los costos y sumando los resultados.
    costo_total = np.sum(distribucion * costos)
    # Calcula la eficiencia total multiplicando la distribución por las eficiencias y sumando los resultados.
    eficiencia_total = np.sum(distribucion * eficiencia)
    # Devuelve la diferencia entre la eficiencia total y el costo total como el valor de aptitud.
    return eficiencia_total - costo_total

# Función para generar una población inicial de soluciones.
def generar_poblacion_inicial(tamano_poblacion, distribucion_inicial):
    poblacion = []
    # Crea una lista de distribuciones iniciales copiadas tamano_poblacion veces.
    for _ in range(tamano_poblacion):
        poblacion.append(distribucion_inicial.copy())
    # Devuelve la población como un array numpy.
    return np.array(poblacion)

# Función para seleccionar los individuos más aptos de la población.
def seleccion(poblacion, fitness_poblacion, num_seleccionados):
    # Ordena los índices de la población en función de los valores de aptitud (fitness) en orden descendente.
    indices_seleccionados = np.argsort(fitness_poblacion)[-num_seleccionados:]
    # Devuelve la subpoblación de los individuos seleccionados.
    return poblacion[indices_seleccionados]

# Función para cruzar (mezclar) dos individuos (padres) y generar dos nuevos individuos (hijos).
def cruce(padre1, padre2):
    # Elige un punto de cruce aleatorio.
    punto_cruce = np.random.randint(1, len(padre1) - 1)
    # Genera dos hijos combinando las partes de los padres a partir del punto de cruce.
    hijo1 = np.concatenate((padre1[:punto_cruce], padre2[punto_cruce:]))
    hijo2 = np.concatenate((padre2[:punto_cruce], padre1[punto_cruce:]))
    # Devuelve los dos hijos.
    return hijo1, hijo2

# Función para mutar (modificar aleatoriamente) un individuo.
def mutacion(individuo, tasa_mutacion=0.01):
    # Itera sobre cada gen en el individuo.
    for i in range(len(individuo)):
        # Si un número aleatorio es menor que la tasa de mutación, cambia el gen a un valor aleatorio.
        if np.random.rand() < tasa_mutacion:
            individuo[i] = np.random.randint(1, 10)  # Ejemplo de mutación
    # Devuelve el individuo mutado.
    return individuo

# Función principal del ciclo del algoritmo genético.
def ciclo_algoritmo_genetico(tamano_poblacion, num_generaciones, tasa_cruce, tasa_mutacion):
    # Genera una distribución inicial aleatoria.
    distribucion_inicial = np.random.randint(1, 10, size=len(costos))
    # Genera la población inicial basada en la distribución inicial.
    poblacion = generar_poblacion_inicial(tamano_poblacion, distribucion_inicial)
    # Itera sobre el número de generaciones especificado.
    for _ in range(num_generaciones):
        # Calcula la aptitud (fitness) de cada individuo en la población.
        fitness_poblacion = np.array([calcular_fitness(ind) for ind in poblacion])
        # Selecciona la mitad superior de la población basada en la aptitud.
        poblacion = seleccion(poblacion, fitness_poblacion, tamano_poblacion // 2)
        nueva_poblacion = []
        # Genera una nueva población mediante cruces y mutaciones hasta alcanzar el tamaño de la población inicial.
        while len(nueva_poblacion) < tamano_poblacion:
            # Selecciona dos padres aleatorios de la población seleccionada.
            padre1, padre2 = poblacion[np.random.choice(poblacion.shape[0], 2, replace=False)]
            # Realiza un cruce con una probabilidad igual a la tasa de cruce.
            if np.random.rand() < tasa_cruce:
                hijo1, hijo2 = cruce(padre1, padre2)
            else:
                hijo1, hijo2 = padre1.copy(), padre2.copy()
            # Realiza una mutación en los hijos con la tasa de mutación especificada.
            hijo1 = mutacion(hijo1, tasa_mutacion)
            hijo2 = mutacion(hijo2, tasa_mutacion)
            # Añade los hijos a la nueva población.
            nueva_poblacion.append(hijo1)
            nueva_poblacion.append(hijo2)
        # Reemplaza la población antigua con la nueva.
        poblacion = np.array(nueva_poblacion)
    # Devuelve la población final después de todas las generaciones.
    return poblacion


# Lógica Difusa

Este código implementa un sistema de control difuso para ajustar la intensidad de las luces en función de la hora del día, el nivel de tráfico y las condiciones climáticas. Se definen variables difusas para cada uno de estos factores y se establecen funciones de pertenencia para categorizar los datos en términos como "madrugada", "tráfico alto" o "clima lluvioso". Luego, se crean reglas difusas para determinar la intensidad de las luces según las combinaciones de estas categorías. La función simular_intensidad toma valores específicos de hora, tráfico y clima, ejecuta la simulación y devuelve la intensidad de luz recomendada en un formato fácil de entender.








In [37]:

# Definiciones del Control Difuso
# Crea una variable difusa (antecedente) para la hora del día, con un rango de 0 a 23.
hora = ctrl.Antecedent(np.arange(0, 24, 1), 'hora')

# Crea una variable difusa (antecedente) para el nivel de tráfico, con un rango de 0 a 100.
trafico = ctrl.Antecedent(np.arange(0, 101, 1), 'trafico')

# Crea una variable difusa (antecedente) para el clima, con un rango de 0 a 100.
clima = ctrl.Antecedent(np.arange(0, 101, 1), 'clima')

# Crea una variable difusa (consecuente) para la intensidad de las luces, con un rango de 0 a 100.
intensidad = ctrl.Consequent(np.arange(0, 101, 1), 'intensidad')

# Define las funciones de pertenencia para la variable 'hora'.
hora['madrugada'] = fuzz.trapmf(hora.universe, [0, 0, 4, 6])
hora['mañana'] = fuzz.trapmf(hora.universe, [4, 6, 10, 12])
hora['tarde'] = fuzz.trapmf(hora.universe, [10, 12, 16, 18])
hora['noche'] = fuzz.trapmf(hora.universe, [16, 18, 24, 24])

# Define las funciones de pertenencia para la variable 'trafico'.
trafico['bajo'] = fuzz.trapmf(trafico.universe, [0, 0, 25, 50])
trafico['medio'] = fuzz.trapmf(trafico.universe, [25, 50, 75, 100])
trafico['alto'] = fuzz.trapmf(trafico.universe, [50, 75, 100, 100])

# Define las funciones de pertenencia para la variable 'clima'.
clima['despejado'] = fuzz.trapmf(clima.universe, [0, 0, 25, 50])
clima['nublado'] = fuzz.trapmf(clima.universe, [25, 50, 75, 100])
clima['lluvioso'] = fuzz.trapmf(clima.universe, [50, 75, 100, 100])

# Define las funciones de pertenencia para la variable 'intensidad'.
intensidad['baja'] = fuzz.trapmf(intensidad.universe, [0, 0, 25, 50])
intensidad['media'] = fuzz.trapmf(intensidad.universe, [25, 50, 75, 100])
intensidad['alta'] = fuzz.trapmf(intensidad.universe, [50, 75, 100, 100])

# Define las reglas difusas.
rule1 = ctrl.Rule(hora['madrugada'] & trafico['bajo'] & clima['despejado'], intensidad['baja'])
rule2 = ctrl.Rule(hora['madrugada'] & trafico['alto'] & clima['nublado'], intensidad['media'])
rule3 = ctrl.Rule(hora['mañana'] & trafico['medio'] & clima['lluvioso'], intensidad['media'])
rule4 = ctrl.Rule(hora['noche'] & trafico['alto'] & clima['despejado'], intensidad['alta'])
rule5 = ctrl.Rule(hora['tarde'] & trafico['alto'] & clima['lluvioso'], intensidad['alta'])
rule6 = ctrl.Rule(hora['mañana'] & trafico['bajo'] & clima['despejado'], intensidad['baja'])
rule7 = ctrl.Rule(hora['mañana'] & trafico['medio'] & clima['despejado'], intensidad['media'])  # Nueva regla de respaldo

# Crea un sistema de control difuso con las reglas definidas.
intensity_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])

# Crea una simulación del sistema de control difuso.
intensity_simulation = ctrl.ControlSystemSimulation(intensity_ctrl)

# Función para formatear la hora de 24 horas a 12 horas (AM/PM).
def formato_hora(hora_24):
    return datetime.strptime(str(hora_24), "%H").strftime("%I:%M %p")

# Función para simular la intensidad de las luces basada en la hora, tráfico y clima.
def simular_intensidad(hora_val, trafico_val, clima_val):
    # Asigna los valores de entrada a la simulación.
    intensity_simulation.input['hora'] = hora_val
    intensity_simulation.input['trafico'] = trafico_val
    intensity_simulation.input['clima'] = clima_val
    try:
        # Ejecuta la simulación para calcular la salida.
        intensity_simulation.compute()
        # Obtiene el valor de la intensidad calculada.
        intensidad_val = intensity_simulation.output['intensidad']
    except ValueError:
        # En caso de error, asigna un valor por defecto de 0 a la intensidad.
        intensidad_val = 0

    # Convierte la hora de 24 horas a formato AM/PM.
    hora_formato = formato_hora(hora_val)
    # Crea un diccionario con los resultados.
    resultado = {
        "hora": hora_formato,
        "trafico": trafico_val,
        "clima": clima_val,
        "intensidad": intensidad_val
    }
    # Devuelve el diccionario con los resultados.
    return resultado


#Lógica Proposicional y Primer Orden




Este código define un conjunto de reglas de lógica proposicional y de primer orden para controlar las acciones de los vehículos en función de las señales de un semáforo. Se crean símbolos que representan los colores del semáforo (verde, amarillo, rojo) y las acciones de los vehículos (detenerse, avanzar, disminuir velocidad). Luego, se establecen reglas lógicas: si el semáforo está en verde, los vehículos avanzan; si está en amarillo, disminuyen la velocidad; y si está en rojo, se detienen. La función evaluar_accion se utiliza para verificar si una acción es posible dada una situación específica y un conjunto de conocimientos, asegurando que la acción no sea contradictoria con las reglas establecidas.








In [38]:

# Lógica Proposicional y Primer Orden
verde, amarillo, rojo = symbols('verde amarillo rojo')  # Define los símbolos que representan los colores del semáforo.
detenerse, avanzar, disminuir_velocidad = symbols('detenerse avanzar disminuir_velocidad')  # Define los símbolos que representan las acciones de los vehículos.
VeSemaforo_C_S, Semaforo_S_rojo, Semaforo_S_verde, Semaforo_S_amarillo = symbols('VeSemaforo_C_S Semaforo_S_rojo Semaforo_S_verde Semaforo_S_amarillo')  # Define símbolos adicionales para las señales del semáforo.
Detenerse_C, Avanzar_C, DisminuirVelocidad_C = symbols('Detenerse_C Avanzar_C DisminuirVelocidad_C')  # Define símbolos adicionales para las acciones correspondientes.

reglas = [
    Implies(verde, avanzar),  # Define la regla: si el semáforo está en verde, los vehículos deben avanzar.
    Implies(amarillo, disminuir_velocidad),  # Define la regla: si el semáforo está en amarillo, los vehículos deben disminuir la velocidad.
    Implies(rojo, detenerse)  # Define la regla: si el semáforo está en rojo, los vehículos deben detenerse.
]

def evaluar_accion(acc, conocimiento):
    return satisfiable(And(conocimiento, acc)) and not satisfiable(And(conocimiento, Not(acc)))  # Evalúa si una acción es posible dado un conjunto de conocimientos, devolviendo True si es satisfacible y no contradictoria.


#Sistema de Producción


Este código define un sistema de producción para gestionar un sistema de agua, modelando sus componentes y estados. La clase SistemaAgua inicializa y almacena el estado del suministro de agua, válvulas, bombas y sensores. Las funciones abrir_valvula, cerrar_valvula, encender_bomba, apagar_bomba y revisar_sensor permiten cambiar el estado de estos componentes. La función diagnosticar_y_reparar realiza diagnósticos y reparaciones automáticas: si el suministro de agua está "sin agua" y todas las válvulas están cerradas, abre todas las válvulas; si hay "baja presión" y alguna bomba no está funcionando, enciende las bombas no funcionales; y si algún sensor no está funcionando, revisa y activa esos sensores.







In [39]:
# Sistema de Producción
class SistemaAgua:  # Define la clase SistemaAgua para modelar el sistema de gestión de agua.
    def __init__(self, estado_suministro, estado_valvulas, estado_bombas, estado_sensores):
        self.estado_suministro = estado_suministro  # Inicializa el estado del suministro de agua.
        self.estado_valvulas = estado_valvulas  # Inicializa el estado de las válvulas.
        self.estado_bombas = estado_bombas  # Inicializa el estado de las bombas.
        self.estado_sensores = estado_sensores  # Inicializa el estado de los sensores.

def abrir_valvula(sistema, valvula):
    sistema.estado_valvulas[valvula] = "abierta"  # Cambia el estado de la válvula especificada a "abierta".

def cerrar_valvula(sistema, valvula):
    sistema.estado_valvulas[valvula] = "cerrada"  # Cambia el estado de la válvula especificada a "cerrada".

def encender_bomba(sistema, bomba):
    sistema.estado_bombas[bomba] = "funcionando"  # Cambia el estado de la bomba especificada a "funcionando".

def apagar_bomba(sistema, bomba):
    sistema.estado_bombas[bomba] = "no funcionando"  # Cambia el estado de la bomba especificada a "no funcionando".

def revisar_sensor(sistema, sensor):
    sistema.estado_sensores[sensor] = "funcionando"  # Cambia el estado del sensor especificado a "funcionando".

def diagnosticar_y_reparar(sistema):
    # Si el suministro de agua está en "sin agua" y todas las válvulas están "cerradas", abre todas las válvulas.
    if sistema.estado_suministro == "sin agua" and all(v == "cerrada" for v in sistema.estado_valvulas.values()):
        for valvula in sistema.estado_valvulas:
            abrir_valvula(sistema, valvula)
    # Si el suministro de agua está en "baja presión" y alguna bomba no está funcionando, enciende las bombas que no están funcionando.
    elif sistema.estado_suministro == "baja presión" and any(b == "no funcionando" for b in sistema.estado_bombas.values()):
        for bomba in sistema.estado_bombas:
            if sistema.estado_bombas[bomba] == "no funcionando":
                encender_bomba(sistema, bomba)
    # Si algún sensor no está funcionando, revisa y activa los sensores que no están funcionando.
    elif any(s == "no funcionando" for s in sistema.estado_sensores.values()):
        for sensor in sistema.estado_sensores:
            if sistema.estado_sensores[sensor] == "no funcionando":
                revisar_sensor(sistema, sensor)


# Redes Semánticas

Este código crea y maneja una representación de la infraestructura de una ciudad usando un grafo. Añade varios elementos como carreteras, tuberías, cables eléctricos y estaciones de bombeo, cada uno con atributos específicos como estado y capacidad. También define las conexiones entre estos elementos, indicando cómo están relacionados entre sí, por ejemplo, si están cerca o si un cable eléctrico alimenta una subestación. Además, incluye funciones para mostrar la información y las conexiones de un elemento específico, y para encontrar el camino más corto entre dos elementos en la infraestructura.

In [40]:

# Definición de la infraestructura
infraestructura = nx.Graph()  # Crea un nuevo grafo vacío para representar la infraestructura.

# Agregar nodos y aristas a la infraestructura (ejemplo)
infraestructura.add_node("Carretera1", tipo="carretera", estado="bueno", longitud=10)  # Añade un nodo representando una carretera con atributos específicos.
infraestructura.add_node("Carretera2", tipo="carretera", estado="regular", longitud=15)  # Añade un segundo nodo para otra carretera con diferentes atributos.
infraestructura.add_node("Tuberia1", tipo="tuberia", estado="malo", capacidad=100)  # Añade un nodo representando una tubería con atributos específicos.
infraestructura.add_node("Tuberia2", tipo="tuberia", estado="bueno", capacidad=150)  # Añade otro nodo para una segunda tubería con diferentes atributos.
infraestructura.add_node("CableElectrico1", tipo="cable electrico", estado="bueno", capacidad=500)  # Añade un nodo para un cable eléctrico con atributos específicos.
infraestructura.add_node("CableElectrico2", tipo="cable electrico", estado="regular", capacidad=400)  # Añade otro nodo para un segundo cable eléctrico con diferentes atributos.
infraestructura.add_node("EstacionBombeo1", tipo="estacion de bombeo", estado="bueno")  # Añade un nodo para una estación de bombeo con atributos específicos.
infraestructura.add_node("SubestacionElectrica1", tipo="subestacion electrica", estado="malo")  # Añade un nodo para una subestación eléctrica con atributos específicos.

# Añade aristas (conexiones) entre los nodos con atributos que describen la relación entre ellos.
infraestructura.add_edge("Carretera1", "Tuberia1", relacion="cercano")  # Conecta Carretera1 con Tuberia1 indicando que están cerca.
infraestructura.add_edge("Carretera1", "CableElectrico1", relacion="cruza")  # Conecta Carretera1 con CableElectrico1 indicando que se cruzan.
infraestructura.add_edge("Carretera2", "Tuberia2", relacion="cercano")  # Conecta Carretera2 con Tuberia2 indicando que están cerca.
infraestructura.add_edge("Carretera2", "CableElectrico2", relacion="cruza")  # Conecta Carretera2 con CableElectrico2 indicando que se cruzan.
infraestructura.add_edge("Tuberia1", "EstacionBombeo1", relacion="conecta")  # Conecta Tuberia1 con EstacionBombeo1 indicando que están conectados.
infraestructura.add_edge("CableElectrico1", "SubestacionElectrica1", relacion="alimenta")  # Conecta CableElectrico1 con SubestacionElectrica1 indicando que la alimenta.
infraestructura.add_edge("CableElectrico2", "SubestacionElectrica1", relacion="alimenta")  # Conecta CableElectrico2 con SubestacionElectrica1 indicando que la alimenta.

# Función para mostrar la información de un nodo específico.
def mostrar_info_nodo(nodo):
    return infraestructura.nodes[nodo]  # Devuelve los atributos del nodo especificado.

# Función para mostrar las conexiones de un nodo específico.
def mostrar_conexiones_nodo(nodo):
    return list(infraestructura.neighbors(nodo))  # Devuelve una lista de los nodos conectados al nodo especificado.

# Función para encontrar el camino más corto entre dos nodos.
def camino_mas_corto(nodo1, nodo2):
    return nx.shortest_path(infraestructura, source=nodo1, target=nodo2)  # Devuelve el camino más corto entre los dos nodos especificados utilizando el algoritmo de caminos más cortos de NetworkX.


# Teoría de Juegos de Nash


Este código crea un juego de teoría de juegos utilizando una matriz de pagos simple, que representa un juego de suma cero similar al dilema del prisionero. Luego, calcula los equilibrios de Nash, que son situaciones donde ningún jugador puede mejorar su resultado cambiando solo su estrategia. Finalmente, imprime todos los equilibrios de Nash encontrados. Esto ayuda a entender las estrategias óptimas que los jugadores deben seguir para mantener el equilibrio en el juego.

In [41]:

# Ejemplo de teoría de juegos de Nash
juego = nash.Game([[1, -1], [-1, 1]])  # Crea un juego utilizando una matriz de pagos. En este caso, es una matriz para un juego de suma cero como el dilema del prisionero.

equilibrios = list(juego.support_enumeration())  # Encuentra los equilibrios de Nash utilizando el método de enumeración de soportes y los almacena en una lista.

# Mostrar equilibrios de Nash
for eq in equilibrios:  # Itera sobre cada equilibrio de Nash encontrado.
    print(f"Equilibrio de Nash: {eq}")  # Imprime cada equilibrio de Nash.


Equilibrio de Nash: (array([0.5, 0.5]), array([0.5, 0.5]))


# Impresión de resultados.

Este código crea una interfaz interactiva para simular y mostrar recomendaciones basadas en varios parámetros como la hora del día, el tráfico y el clima. También gestiona el estado de un sistema de agua y la infraestructura de una ciudad. Los usuarios pueden ajustar estos parámetros mediante controles deslizantes y menús desplegables, y el sistema actualizará automáticamente las recomendaciones y mostrará gráficos que visualizan la distribución de autobuses, la intensidad de las luces y el estado del sistema de agua e infraestructura.

In [42]:
# Crear widgets
ciudad_text = widgets.Text(value='Ancud', description='Ciudad:', disabled=False)  # Crea un widget de texto para ingresar la ciudad, con valor inicial 'Ancud'.

# Crear salidas
salida_resultados = widgets.Output()  # Crea un área de salida para mostrar los resultados.

# Función para actualizar recomendaciones
def actualizar_recomendaciones(change=None):
    ciudad = ciudad_text.value  # Obtiene el valor ingresado en el widget de texto de la ciudad.
    hora_val = hora_slider.value  # Obtiene el valor seleccionado en el slider de hora.
    trafico_val = trafico_slider.value  # Obtiene el valor seleccionado en el slider de tráfico.
    clima_val = {'Favorable': 20, 'Normal': 50, 'Desfavorable': 80}[clima_dropdown.value]  # Traduce el valor seleccionado en el desplegable de clima a un valor numérico.

    # Simulación de intensidad
    resultado_intensidad = simular_intensidad(hora_val, trafico_val, clima_val)  # Simula la intensidad de las luces basada en los valores de hora, tráfico y clima.

    # Diagnóstico del sistema de agua
    estado_suministro = estado_suministro_selector.value  # Obtiene el valor seleccionado en el desplegable de estado de suministro.
    estado_valvulas = {k: v.value for k, v in estado_valvulas_selectors.items()}  # Crea un diccionario con los estados de las válvulas.
    estado_bombas = {k: v.value for k, v in estado_bombas_selectors.items()}  # Crea un diccionario con los estados de las bombas.
    estado_sensores = {k: v.value for k, v in estado_sensores_selectors.items()}  # Crea un diccionario con los estados de los sensores.

    sistema_agua = SistemaAgua(estado_suministro, estado_valvulas, estado_bombas, estado_sensores)  # Inicializa un objeto SistemaAgua con los estados actuales.
    diagnosticar_y_reparar(sistema_agua)  # Diagnostica y repara el sistema de agua según los estados actuales.

    # Generar la salida formateada
    with salida_resultados:  # Utiliza el área de salida creada para mostrar los resultados.
        salida_resultados.clear_output()  # Limpia la salida anterior.

        # Muestra y grafica la distribución optimizada de autobuses
        print(f"Distribución optimizada de autobuses en {ciudad}:")
        distribucion_optimizada = ciclo_algoritmo_genetico(10, 100, 0.8, 0.01)  # Ejecuta el algoritmo genético para optimizar la distribución de autobuses.
        print(f"Fitness del plan de distribución: {calcular_fitness(distribucion_optimizada[0])}")
        for zona, num_autobuses in zip(zonas, distribucion_optimizada[0]):  # Itera sobre las zonas y el número de autobuses.
            print(f"  - Zona: {zona}, Número de autobuses: {num_autobuses}")

        # Gráfico de la distribución optimizada de autobuses
        fig, ax = plt.subplots()  # Crea una figura y un eje para el gráfico.
        ax.bar(zonas, distribucion_optimizada[0])  # Crea un gráfico de barras con las zonas y el número de autobuses.
        ax.set_xlabel('Zonas')  # Etiqueta el eje X.
        ax.set_ylabel('Número de autobuses')  # Etiqueta el eje Y.
        ax.set_title(f'Distribución optimizada de autobuses en {ciudad}')  # Título del gráfico.
        plt.xticks(rotation=45)  # Rota las etiquetas del eje X 45 grados.
        plt.show()  # Muestra el gráfico.

        # Muestra y grafica las recomendaciones de intensidad de luces
        print(f"\nRecomendaciones de intensidad de luces en {ciudad} según simulaciones:")
        print(f"  - Hora del día: {resultado_intensidad['hora']}")
        print(f"  - Nivel de tráfico: {resultado_intensidad['trafico']}%")
        print(f"  - Condición climática: {resultado_intensidad['clima']}%")
        print(f"  - Intensidad de luces recomendada: {resultado_intensidad['intensidad']:.2f}%")

        # Gráfico de la intensidad de luces recomendada
        labels = ['Hora del día', 'Nivel de tráfico', 'Condición climática']  # Define las etiquetas para el gráfico.
        values = [hora_val, trafico_val, clima_val]  # Define los valores para el gráfico.
        fig, ax = plt.subplots()  # Crea una figura y un eje para el gráfico.
        ax.bar(labels, values)  # Crea un gráfico de barras con las etiquetas y los valores.
        ax.set_ylabel('Valor')  # Etiqueta el eje Y.
        ax.set_title(f'Recomendaciones de intensidad de luces en {ciudad}')  # Título del gráfico.
        plt.show()  # Muestra el gráfico.

        # Muestra los resultados de la lógica proposicional y de primer orden
        print(f"\nResultados de la lógica proposicional para el control de tráfico en {ciudad}:")
        if evaluar_accion(avanzar, verde):  # Evalúa si la acción de avanzar es posible con el semáforo en verde.
            print("  - El semáforo está en verde: los vehículos pueden avanzar.")

        print(f"\nResultados de la lógica de primer orden para el control de tráfico en {ciudad}:")
        if evaluar_accion(avanzar, verde):  # Evalúa si la acción de avanzar es posible con el semáforo en verde.
            print("  - En base a la lógica de primer orden, los vehículos pueden avanzar.")

        # Gráfico de estados inicial y final de los sistemas de agua
        estado_inicial = [estado_suministro_selector.value,
                          estado_valvulas_selectors['valvula1'].value,
                          estado_valvulas_selectors['valvula2'].value,
                          estado_bombas_selectors['bomba1'].value,
                          estado_bombas_selectors['bomba2'].value,
                          estado_sensores_selectors['sensor1'].value,
                          estado_sensores_selectors['sensor2'].value]

        diagnosticar_y_reparar(sistema_agua)  # Actualiza el diagnóstico y reparación del sistema de agua.

        estado_final = [sistema_agua.estado_suministro,
                        sistema_agua.estado_valvulas['valvula1'],
                        sistema_agua.estado_valvulas['valvula2'],
                        sistema_agua.estado_bombas['bomba1'],
                        sistema_agua.estado_bombas['bomba2'],
                        sistema_agua.estado_sensores['sensor1'],
                        sistema_agua.estado_sensores['sensor2']]

        print(f"\nEstados inicial y final de los sistemas de agua en {ciudad}:")
        print(f"  - Estado inicial: {estado_inicial}")
        print(f"  - Estado final: {estado_final}")

        labels = ['Suministro', 'Válvula 1', 'Válvula 2', 'Bomba 1', 'Bomba 2', 'Sensor 1', 'Sensor 2']  # Define las etiquetas para el gráfico.
        fig, ax = plt.subplots()  # Crea una figura y un eje para el gráfico.
        ax.bar(labels, estado_inicial, alpha=0.5, label='Inicial')  # Crea un gráfico de barras con el estado inicial.
        ax.bar(labels, estado_final, alpha=0.5, label='Final')  # Crea un gráfico de barras con el estado final.
        ax.set_ylabel('Estado')  # Etiqueta el eje Y.
        ax.set_title(f'Estados inicial y final de los sistemas de agua en {ciudad}')  # Título del gráfico.
        ax.legend()  # Muestra la leyenda del gráfico.
        plt.xticks(rotation=45)  # Rota las etiquetas del eje X 45 grados.
        plt.show()  # Muestra el gráfico.

        # Muestra y grafica la información detallada sobre la infraestructura
        print(f"\nInformación detallada sobre la infraestructura en {ciudad}:")
        nodo1_info = mostrar_info_nodo('Carretera1')  # Obtiene la información del nodo 'Carretera1'.
        nodo1_conexiones = mostrar_conexiones_nodo('Carretera1')  # Obtiene las conexiones del nodo 'Carretera1'.
        nodo1_camino = camino_mas_corto('Carretera1', 'SubestacionElectrica1')  # Obtiene el camino más corto entre 'Carretera1' y 'SubestacionElectrica1'.
        print(f"Información sobre 'Carretera1':\n  - Tipo: {nodo1_info['tipo']}\n  - Estado: {nodo1_info['estado']}\n  - Longitud: {nodo1_info['longitud']} km\n  - Conexiones: {', '.join(nodo1_conexiones)}\n  - Camino más corto entre 'Carretera1' y 'SubestacionElectrica1': {', '.join(nodo1_camino)}")

        nodo2_info = mostrar_info_nodo('Tuberia1')  # Obtiene la información del nodo 'Tuberia1'.
        nodo2_conexiones = mostrar_conexiones_nodo('Tuberia1')  # Obtiene las conexiones del nodo 'Tuberia1'.
        nodo2_camino = camino_mas_corto('Tuberia1', 'SubestacionElectrica1')  # Obtiene el camino más corto entre 'Tuberia1' y 'SubestacionElectrica1'.
        print(f"\nInformación sobre 'Tuberia1':\n  - Tipo: {nodo2_info['tipo']}\n  - Estado: {nodo2_info['estado']}\n  - Capacidad: {nodo2_info['capacidad']} L\n  - Conexiones: {', '.join(nodo2_conexiones)}\n  - Camino más corto entre 'Tuberia1' y 'SubestacionElectrica1': {', '.join(nodo2_camino)}")

        # Gráfico de la infraestructura
        labels = ['Carretera1', 'Tuberia1', 'CableElectrico1', 'EstacionBombeo1', 'SubestacionElectrica1']  # Define las etiquetas para el gráfico.
        estados = [nodo1_info['estado'], nodo2_info['estado'],
                   mostrar_info_nodo('CableElectrico1')['estado'],
                   mostrar_info_nodo('EstacionBombeo1')['estado'],
                   mostrar_info_nodo('SubestacionElectrica1')['estado']]  # Obtiene los estados de los nodos de infraestructura.
        capacidades = [nodo1_info.get('longitud', 0), nodo2_info.get('capacidad', 0),
                       mostrar_info_nodo('CableElectrico1').get('capacidad', 0),
                       mostrar_info_nodo('EstacionBombeo1').get('capacidad', 0),
                       mostrar_info_nodo('SubestacionElectrica1').get('capacidad', 0)]  # Obtiene las capacidades de los nodos de infraestructura.

        fig, ax = plt.subplots()  # Crea una figura y un eje para el gráfico.
        ax.bar(labels, capacidades)  # Crea un gráfico de barras con las etiquetas y las capacidades.
        ax.set_ylabel('Capacidad / Longitud')  # Etiqueta el eje Y.
        ax.set_title(f'Capacidades / Longitudes de la infraestructura en {ciudad}')  # Título del gráfico.
        plt.xticks(rotation=45)  # Rota las etiquetas del eje X 45 grados.
        plt.show()  # Muestra el gráfico.

        fig, ax = plt.subplots()  # Crea una figura y un eje para el gráfico.
        ax.bar(labels, [1 if estado == 'bueno' else 0.5 if estado == 'regular' else 0 for estado in estados])  # Crea un gráfico de barras con los estados de los nodos.
        ax.set_ylabel('Estado (1: bueno, 0.5: regular, 0: malo)')  # Etiqueta el eje Y.
        ax.set_title(f'Estados de la infraestructura en {ciudad}')  # Título del gráfico.
        plt.xticks(rotation=45)  # Rota las etiquetas del eje X 45 grados.
        plt.show()  # Muestra el gráfico.

# Enlazar widgets con la función de actualización
hora_slider.observe(actualizar_recomendaciones, names='value')  # Enlaza el slider de hora con la función de actualización.
trafico_slider.observe(actualizar_recomendaciones, names='value')  # Enlaza el slider de tráfico con la función de actualización.
clima_dropdown.observe(actualizar_recomendaciones, names='value')  # Enlaza el desplegable de clima con la función de actualización.
estado_suministro_selector.observe(actualizar_recomendaciones, names='value')  # Enlaza el desplegable de estado de suministro con la función de actualización.
for selector in estado_valvulas_selectors.values():
    selector.observe(actualizar_recomendaciones, names='value')  # Enlaza los desplegables de estado de válvulas con la función de actualización.
for selector in estado_bombas_selectors.values():
    selector.observe(actualizar_recomendaciones, names='value')  # Enlaza los desplegables de estado de bombas con la función de actualización.
for selector in estado_sensores_selectors.values():
    selector.observe(actualizar_recomendaciones, names='value')  # Enlaza los desplegables de estado de sensores con la función de actualización.
ciudad_text.observe(actualizar_recomendaciones, names='value')  # Enlaza el widget de texto de la ciudad con la función de actualización.

# Mostrar widgets adicionales y salidas
display(ciudad_text)  # Muestra el widget de texto de la ciudad.
display(salida_resultados)  # Muestra el área de salida para los resultados.

# Llamar a la función inicialmente para mostrar la salida
actualizar_recomendaciones()  # Llama a la función de actualización para mostrar los resultados iniciales.


Text(value='Ancud', description='Ciudad:')

Output()

# Repositorio GitHub

https://github.com/michaelarjelm/evaluacionTresIA/blob/main/Evaluacion_3.ipynb